# Requirements

In [1]:
!pip install --user gdelt

# Import modules

In [1]:
from datetime import date, timedelta
from gdelt import gdelt as gdelt_client
import matplotlib.pyplot as plot
import pandas
import seaborn
import tempfile

# Query the knowledge graph
Use coverage option for querying all daily records. Otherwise records collected from the last 15 minutes are returned.
Use the date option to filter by date.

In [2]:
def get_graph(date, coverage=False):
    client = gdelt_client(version=2)
    graph = client.Search(date.strftime("%Y %m %d"), table="gkg", coverage=coverage)
    graph = graph.astype({"DATE": str})
    graph["DATE"] = graph["DATE"].apply(lambda dateStr: dateStr[:14])
    del client
    return graph

def get_today_graph(coverage=False):
    return get_graph(date.today(), coverage)

def get_yesterday_graph(coverage=False):
    return get_graph(date.today()-timedelta(days=1), coverage)

In [3]:
report_date = date.today()-timedelta(days=1)
graph = get_graph(report_date, coverage=True)
corona_records = graph.loc[graph["V2Themes"].str.contains("TAX_DISEASE_CORONAVIRUS", na=False)]

# Save records to temp

In [4]:
corona_records.to_csv("{}/corona_{}.csv".format(tempfile.gettempdir(), report_date.strftime("%Y%m%d")), index=False)

# Location type exploding and filtering

In [5]:
from enum import Enum

class location_type(Enum):
    """Location type
        Defines the different location types.
    """
    UNKNOWN = 0
    COUNTRY = 1
    USSTATE = 2
    USCITY = 3
    WORLDCITY = 4
    WORLDSTATE = 5

class gdelt_location:
    """GDELT location
        Defines a GDELT location.
    """
    def __init__(self, location_typeid=0, name=None, country_code=None, admin1_code=None, lat=None, lon=None, feature_id=None):        
        self.location_type = location_type(int(location_typeid))
        self.location_name = name
        self.country_code = country_code
        self.admin1_code = admin1_code
        self.location_lat = lat
        self.location_lon = lon
        self.feature_id = feature_id
        
    def has_location_type(self, location_type):
        return location_type == self.location_type
    
    def location_type_matches(self, location_types):
        return self.location_type in location_types
    
    def __str__(self):
        return self.location_name
    
class location_filter():
    """Location Filter
        Defines different filters which can be applied on the dataframes.
    """
    def filter_by_type(self, gkg_dataframe, location_type):
        return gkg_dataframe.loc[gkg_dataframe.apply(lambda record: record["GDELT_Locations"].has_location_type(location_type), axis=1)]
    
    def filter_by_types(self, gkg_dataframe, location_types):
        return gkg_dataframe.loc[gkg_dataframe.apply(lambda record: record["GDELT_Locations"].location_type_matches(location_types), axis=1)]

def split_location_entries(locations):
    return [gdelt_location(*location) if 7 == len(location) else gdelt_location() for location in locations]

def split_locations(record):
    return split_location_entries([location.split("#") for location in str(record["Locations"]).split(";")])


# Filter the locations by location type

In [6]:
corona_locations = corona_records.copy(deep=True)
corona_locations["GDELT_Locations"] = corona_records.apply(lambda record: split_locations(record), axis=1)
corona_locations_exploded = corona_locations.explode("GDELT_Locations")

filter = location_filter()
corona_filtered_locations = filter.filter_by_types(corona_locations_exploded, [location_type.WORLDCITY, location_type.USCITY])
del corona_records
del graph

# Extract the coordinates and the name from the GDELT location

In [7]:
def to_point_locations(gkg_dataframe):
    if gkg_dataframe.empty:
        return pandas.DataFrame(columns=["GKGRECORDID", 
                                         "DATE",
                                         "SourceCommonName",
                                         "DocumentIdentifier",
                                         "Location_Name",
                                         "Location_Lat",
                                         "Location_Lon"])
        
    point_locations = gkg_dataframe[["GKGRECORDID", 
                                                 "DATE",
                                                 "SourceCommonName",
                                                 "DocumentIdentifier",
                                                 "GDELT_Locations"]].copy(deep=True)
    point_locations["Location_Name"] = gkg_dataframe.apply(lambda record: record["GDELT_Locations"].location_name, axis=1)
    point_locations["Location_Lat"] = gkg_dataframe.apply(lambda record: record["GDELT_Locations"].location_lat, axis=1)
    point_locations["Location_Lon"] = gkg_dataframe.apply(lambda record: record["GDELT_Locations"].location_lon, axis=1)
    return point_locations.drop("GDELT_Locations", axis=1)

corona_point_locations = to_point_locations(corona_filtered_locations)
del corona_filtered_locations

corona_point_locations

,GKGRECORDID,DATE,SourceCommonName,DocumentIdentifier,Location_Name,Location_Lat,Location_Lon
1,20200314000000-1,20200314000000,mykxlg.com,https://www.mykxlg.com/news/national/on-eve-of...,"Hollywood, California, United States",34.0983,-118.327
1,20200314000000-1,20200314000000,mykxlg.com,https://www.mykxlg.com/news/national/on-eve-of...,"Anaheim, California, United States",33.8353,-117.915
3,20200314000000-3,20200314000000,newportri.com,https://www.newportri.com/zz/news/20200313/sch...,"Philadelphia, Pennsylvania, United States",39.9523,-75.1638
3,20200314000000-3,20200314000000,newportri.com,https://www.newportri.com/zz/news/20200313/sch...,"Cleveland, Ohio, United States",41.4995,-81.6954
3,20200314000000-3,20200314000000,newportri.com,https://www.newportri.com/zz/news/20200313/sch...,"Valley High School, California, United States",33.7231,-117.901
...,...,...,...,...,...,...,...
121331,20200314234500-914,20200314234500,kgw.com,https://www.kgw.com/article/news/health/corona...,"Washington, Washington, United States",38.8951,-77.0364
121331,20200314234500-914,20200314234500,kgw.com,https://www.kgw.com/article/news/health/corona...,"Linn County, Oregon, United States",44.4996,-122.585
121336,20200314234500-919,20200314234500,guardian-series.co.uk,https://www.guardian-series.co.uk/news/nationa...,"London, London, City Of, United Kingdom",51.5,-0.116667
121339,20200314234500-922,20200314234500,790wpic.com,https://www.790wpic.com/news/trump-says-he-was...,"Washington, Washington, United States",38.8951,-77.0364


# Save point locations to temp

In [8]:
corona_point_locations.to_csv("{}/corona_locations_{}.gkg.csv".format(tempfile.gettempdir(), report_date.strftime("%Y%m%d")), index=False)

del corona_point_locations